In [ ]:
import timeit
import numpy as np
from scipy import stats, interpolate
import matplotlib.pyplot as plt
import gudhi as gd
import gudhi.representations as gdr
from alpha_complex_periodic import calc_persistence
from gadgetutils import utils

In [ ]:
plt.style.use(["science", "notebook"])

# Timing tests

In [ ]:
N_vals = np.logspace(1.9, 4.6, 12, dtype=int)

In [ ]:
def pers_reg(points, precision="safe"):
    st = gd.AlphaComplex(points=points, precision=precision).create_simplex_tree()
    st.compute_persistence()
    pairs = [np.sqrt(st.persistence_intervals_in_dimension(d)) for d in range(3)]
    return pairs

In [ ]:
timers_reg = [timeit.Timer("pers_reg(points)", f"points = np.random.rand({N}, 3)", globals=globals()) for N in N_vals]
results_reg = [timer.autorange() for timer in timers_reg]
time_reg = [x[1]/x[0] for x in results_reg]

In [ ]:
timers_reg3d = [timeit.Timer("calc_persistence(points)", f"points = np.random.rand({N}, 3)", globals=globals()) for N in N_vals]
results_reg3d = [timer.autorange() for timer in timers_reg3d]
time_reg3d = [x[1]/x[0] for x in results_reg3d]

In [ ]:
timers_per = [timeit.Timer("calc_persistence(points, boxsize=1)", f"points = np.random.rand({N}, 3)", globals=globals()) for N in N_vals]
results_per = [timer.autorange() for timer in timers_per]
time_per = [x[1]/x[0] for x in results_per]

In [ ]:
plt.plot(N_vals, time_reg, 'o', label="regular triangulation")
plt.plot(N_vals, time_reg3d, 'o', label="regular (3D) triangulation")
plt.plot(N_vals, time_per, 'o', label="periodic triangulation")
plt.loglog()
plt.xlabel("$N_{points}$")
plt.ylabel("runtime  [s]")
plt.legend(frameon=True)
plt.show()

In [ ]:
fit_reg = stats.linregress(np.log(N_vals), np.log(time_reg))
fit_per = stats.linregress(np.log(N_vals)[4:], np.log(time_per)[4:])

print(f"Slope for regular triangulation: {fit_reg.slope:.2f}")
print(f"Slope for periodic triangulation: {fit_per.slope:.2f}")

print()

factor = np.mean(np.array(time_per)[4:] / np.array(time_reg)[4:])
print(f"Periodic triangulation is ~ {factor:.1f} times slower than the regular triangulation (for >500 points)")

### Test different precision levels

Triangulations can be either `SAFE`, `FAST`, or `EXACT`

In [ ]:
timers_reg_fast = [timeit.Timer("pers_reg(points, precision='fast')", f"points = np.random.rand({N}, 3)", globals=globals()) for N in N_vals]
results_reg_fast = [timer.autorange() for timer in timers_reg_fast]
time_reg_fast = [x[1]/x[0] for x in results_reg_fast]

timers_reg_exact = [timeit.Timer("pers_reg(points, precision='exact')", f"points = np.random.rand({N}, 3)", globals=globals()) for N in N_vals]
results_reg_exact = [timer.autorange() for timer in timers_reg_exact]
time_reg_exact = [x[1]/x[0] for x in results_reg_exact]

In [ ]:
timers_per_fast = [timeit.Timer("calc_persistence(points, precision='fast', boxsize=1)", f"points = np.random.rand({N}, 3)", globals=globals()) for N in N_vals]
results_per_fast = [timer.autorange() for timer in timers_per_fast]
time_per_fast = [x[1]/x[0] for x in results_per_fast]

timers_per_exact = [timeit.Timer("calc_persistence(points, precision='exact', boxsize=1)", f"points = np.random.rand({N}, 3)", globals=globals()) for N in N_vals]
results_per_exact = [timer.autorange() for timer in timers_per_exact]
time_per_exact = [x[1]/x[0] for x in results_per_exact]

In [ ]:
plt.figure(figsize=(14,6))
plt.subplot(1,2,1)
plt.plot(N_vals, time_reg, 'o', label="safe")
plt.plot(N_vals, time_reg_fast, 'o', label="fast")
plt.plot(N_vals, time_reg_exact, 'o', label="exact")
plt.loglog()
plt.title("Precision levels for regular triangulation")
plt.xlabel("$N_{points}$")
plt.ylabel("runtime  [s]")
plt.legend(frameon=True)

plt.subplot(1,2,2)
plt.plot(N_vals, time_per, 'o', label="safe")
plt.plot(N_vals, time_per_fast, 'o', label="fast")
plt.plot(N_vals, time_per_exact, 'o', label="exact")
plt.loglog()
plt.title("Precision levels for periodic triangulation")
plt.xlabel("$N_{points}$")
plt.ylabel("runtime  [s]")
plt.legend(frameon=True)
plt.show()

factor_fast = np.mean(np.array(time_per_fast)[4:] / np.array(time_reg_fast)[4:])
print(f"Fast periodic triangulation is ~{factor_fast:.1f} times slower than fast regular triangulation")

Precision levels only affect filtration values.

- `SAFE` - filtration values might have a small multipllicative error (~$10^{-6}$) compared to exact values
- `FAST`- filtration values might be arbitrarily bad

In quick test with uniform random points ($N_{points}$ between 300 and 15,000) the fast results were exactly the same as the exact ones.

# Compare periodic / non-periodic topological summaries

Using random points uniformly distributied inside a box $[0, L)$.

In [ ]:
N = 500
boxsize = 1
lbar = boxsize / np.cbrt(N)
points = boxsize * np.random.rand(N, 3)

In [ ]:
pairs_periodic = calc_persistence(points, boxsize=boxsize)

st = gd.AlphaComplex(points=points).create_simplex_tree()
st.compute_persistence()
pairs_regular = [np.sqrt(st.persistence_intervals_in_dimension(d)) for d in range(3)]

### Persistence diagrams

In [ ]:
plt.figure(figsize=(14,6))
colors = ['b', 'g', 'r']
plt.subplot(1,2,1)
plt.title("periodic triangulation")
for dim in range(3):
    plt.plot(*np.array(pairs_periodic[dim]).T, 'o', alpha=0.6, label=f"$H_{{{dim}}}$")
plt.plot([0, 0.2], [0, 0.2], 'r-')
plt.legend(frameon=True)

plt.subplot(1,2,2)
plt.title("regular triangulation")
for dim in range(3):
    plt.plot(*np.array(pairs_regular[dim]).T, 'o', alpha=0.6, label=f"$H_{{{dim}}}$")
plt.plot([0, 0.2], [0, 0.2], 'r-')
plt.show()

### Weighted silhouttes

In [ ]:
DS = gdr.preprocessing.DiagramSelector(use=True)
SIL = gdr.Silhouette(weight=lambda x: np.power(x[1] - x[0], 1), resolution=500, sample_range=[0, 0.2])
x = np.linspace(*SIL.sample_range, SIL.resolution)

In [ ]:
sils_periodic = SIL.fit_transform(DS.fit_transform([np.array(p) for p in pairs_periodic]))
sils_regular = SIL.fit_transform(DS.fit_transform([np.array(p) for p in pairs_regular]))

In [ ]:
plt.figure(figsize=(22,6))
plt.subplot(1,3,1)
plt.title("$H_0$")
plt.plot(x, sils_periodic[0], label="periodic")
plt.plot(x, sils_regular[0], label="regular")
plt.xlim(0, 0.1)
plt.legend(frameon=True)

plt.subplot(1,3,2)
plt.title("$H_1$")
plt.plot(x, sils_periodic[1], label="periodic")
plt.plot(x, sils_regular[1], label="regular")
plt.xlim(0.02, 0.15)

plt.subplot(1,3,3)
plt.title("$H_2$")
plt.plot(x, sils_periodic[2], label="periodic")
plt.plot(x, sils_regular[2], label="regular")
plt.xlim(0.06, 0.18)
plt.show()

per_peak = x[np.argmax(sils_periodic, axis=1)] / lbar
reg_peak = x[np.argmax(sils_regular, axis=1)] / lbar
print(f"Periodic:  H_0 peak: {per_peak[0]:.3f} lbar\t H_1 peak: {per_peak[1]:.3f} lbar\t H_2 peak: {per_peak[2]:.3f} lbar")
print(f"Regular:   H_0 peak: {reg_peak[0]:.3f} lbar\t H_1 peak: {reg_peak[1]:.3f} lbar\t H_2 peak: {per_peak[2]:.3f} lbar")

Individual curves do not match that well.

Next, generate many samples of regular triangulations by shifting center of periodic box

Compare mean to periodic curve

In [ ]:
n_samples = 50
shifted_centers = [utils.center_box_pbc(points, boxsize*np.random.rand(3), boxsize) + boxsize/2 for i in range(n_samples)]

#### Shifting center should not change periodic triangulation

In [ ]:
pairs_shift_per = [calc_persistence(points, boxsize=boxsize) for points in shifted_centers]

In [ ]:
sils_shift_per = np.array([SIL.fit_transform(DS.fit_transform([np.array(p[i]) for p in pairs_shift_per])) for i in range(3)])
sils_per_mean = np.mean(sils_shift_per, axis=1)
sils_per_std = np.std(sils_shift_per, axis=1)

In [ ]:
colors = ['b', 'g', 'r']
for i in range(n_samples):
    for d in range(3):
        plt.plot(x, sils_shift_per[d,i], color=colors[d], label=f"$H_{d}$" if not i else "")
plt.xlim(0, 0.18)
plt.legend(frameon=True)
plt.title(f"Weighted silhouttes for randomly shifted periodic triangulations")
plt.show()

#### Shifting center does change regular triangulation

In [ ]:
sts = [gd.AlphaComplex(points).create_simplex_tree() for points in shifted_centers]
for sti in sts: sti.compute_persistence()
pairs_shift_reg = [[np.sqrt(sti.persistence_intervals_in_dimension(d)) for sti in sts] for d in range(3)]

In [ ]:
sils_shift_reg = np.array([SIL.fit_transform(DS.fit_transform(p)) for p in pairs_shift_reg])
sils_reg_mean = np.mean(sils_shift_reg, axis=1)
sils_reg_std = np.std(sils_shift_reg, axis=1)

In [ ]:
colors = ['b', 'g', 'r']
for i in range(n_samples):
    for d in range(3):
        plt.plot(x, sils_shift_reg[d,i], color=colors[d], label=f"$H_{d}$" if not i else "")
plt.xlim(0, 0.18)
plt.legend(frameon=True)
plt.title(f"Weighted silhouttes for randomly shifted regular triangulations")
plt.show()

#### Compare means

In [ ]:
plt.figure(figsize=(22,6))
plt.subplot(1,3,1)
plt.title("$H_0$")
plt.plot(x, sils_periodic[0], label="periodic")
plt.plot(x, sils_reg_mean[0], label="regular")
plt.fill_between(x, sils_reg_mean[0]-sils_reg_std[0], sils_reg_mean[0]+sils_reg_std[0], alpha=0.3, color='g')
plt.xlim(0, 0.08)
plt.legend(frameon=True)

plt.subplot(1,3,2)
plt.title("$H_1$")
plt.plot(x, sils_periodic[1], label="periodic")
plt.plot(x, sils_reg_mean[1], label="regular")
plt.fill_between(x, sils_reg_mean[1]-sils_reg_std[1], sils_reg_mean[1]+sils_reg_std[1], alpha=0.3, color='g')
plt.xlim(0.02, 0.16)

plt.subplot(1,3,3)
plt.title("$H_2$")
plt.plot(x, sils_periodic[2], label="periodic")
plt.plot(x, sils_reg_mean[2], label="regular")
plt.fill_between(x, sils_reg_mean[2]-sils_reg_std[2], sils_reg_mean[2]+sils_reg_std[2], alpha=0.3, color='g')
plt.xlim(0.05, 0.2)
plt.show()

per_peak = x[np.argmax(sils_periodic, axis=1)] / lbar
reg_peak = x[np.argmax(sils_reg_mean, axis=1)] / lbar
print(f"Periodic:  H_0 peak: {per_peak[0]:.3f} lbar\t H_1 peak: {per_peak[1]:.3f} lbar\t H_2 peak: {per_peak[2]:.3f} lbar")
print(f"Regular:   H_0 peak: {reg_peak[0]:.3f} lbar\t H_1 peak: {reg_peak[1]:.3f} lbar\t H_2 peak: {per_peak[2]:.3f} lbar")

### Betti curves

In [ ]:
BC = gdr.BettiCurve(resolution=SIL.resolution, sample_range=SIL.sample_range)

In [ ]:
bc_periodic = BC.fit_transform(DS.fit_transform([np.array(p) for p in pairs_periodic]))
bc_regular = BC.fit_transform(DS.fit_transform([np.array(p) for p in pairs_regular]))

In [ ]:
plt.figure(figsize=(22,6))
plt.subplot(1,3,1)
plt.title("$H_0$")
plt.plot(x, bc_periodic[0], label="periodic")
plt.plot(x, bc_regular[0], label="regular")
plt.xlim(0, 0.12)
plt.legend(frameon=True)

plt.subplot(1,3,2)
plt.title("$H_1$")
plt.plot(x, bc_periodic[1], label="periodic")
plt.plot(x, bc_regular[1], label="regular")
plt.xlim(0, 0.18)

plt.subplot(1,3,3)
plt.title("$H_2$")
plt.plot(x, bc_periodic[2], label="periodic")
plt.plot(x, bc_regular[2], label="regular")
plt.xlim(0.05, 0.2)
plt.show()

per_halfmax = x[np.argmin(np.abs(bc_periodic[0] - 0.5*bc_periodic[0][0]))] / lbar
reg_halfmax = x[np.argmin(np.abs(bc_regular[0] - 0.5*bc_regular[0][0]))] / lbar
per_peak = x[np.argmax(bc_periodic[1:], axis=1)] / lbar
reg_peak = x[np.argmax(bc_regular[1:], axis=1)] / lbar
print(f"Periodic:  H_0 halfmax: {per_halfmax:.3f} lbar\t H_1 peak: {per_peak[0]:.3f} lbar\t H_2 peak: {per_peak[1]:.3f} lbar")
print(f"Regular:   H_0 halfmax: {reg_halfmax:.3f} lbar\t H_1 peak: {reg_peak[0]:.3f} lbar\t H_2 peak: {reg_peak[1]:.3f} lbar")

Look at full distribution of regular curves

In [ ]:
bc_samples = [BC.fit_transform(DS.fit_transform(p)) for p in pairs_samples]
bc_reg_mean = [np.mean(b, axis=0) for b in bc_samples]
bc_reg_std = [np.std(b, axis=0) for b in bc_samples]

In [ ]:
plt.figure(figsize=(22,6))
plt.subplot(1,3,1)
plt.title("$H_0$")
plt.plot(x, bc_periodic[0], label="periodic")
plt.plot(x, bc_reg_mean[0], label="regular")
plt.fill_between(x, bc_reg_mean[0]-bc_reg_std[0], bc_reg_mean[0]+bc_reg_std[0], alpha=0.3, color='g')
plt.xlim(0, 0.12)
plt.legend(frameon=True)

plt.subplot(1,3,2)
plt.title("$H_1$")
plt.plot(x, bc_periodic[1], label="periodic")
plt.plot(x, bc_reg_mean[1], label="regular")
plt.fill_between(x, bc_reg_mean[1]-bc_reg_std[1], bc_reg_mean[1]+bc_reg_std[1], alpha=0.3, color='g')
plt.xlim(0, 0.18)

plt.subplot(1,3,3)
plt.title("$H_2$")
plt.plot(x, bc_periodic[2], label="periodic")
plt.plot(x, bc_reg_mean[2], label="regular")
plt.fill_between(x, bc_reg_mean[2]-bc_reg_std[2], bc_reg_mean[2]+bc_reg_std[2], alpha=0.3, color='g')
plt.xlim(0.05, 0.2)
plt.show()

per_halfmax = x[np.argmin(np.abs(bc_periodic[0] - 0.5*bc_periodic[0][0]))] / lbar
reg_halfmax = x[np.argmin(np.abs(bc_reg_mean[0] - 0.5*bc_reg_mean[0][0]))] / lbar
per_peak = x[np.argmax(bc_periodic[1:], axis=1)] / lbar
reg_peak = x[np.argmax(bc_reg_mean[1:], axis=1)] / lbar
print(f"Periodic:  H_0 halfmax: {per_halfmax:.3f} lbar\t H_1 peak: {per_peak[0]:.3f} lbar\t H_2 peak: {per_peak[1]:.3f} lbar")
print(f"Regular:   H_0 halfmax: {reg_halfmax:.3f} lbar\t H_1 peak: {reg_peak[0]:.3f} lbar\t H_2 peak: {reg_peak[1]:.3f} lbar")

### Entropy summary curves

cleaner and provide better comparisons than the Betti curves

In [ ]:
ES = gdr.Entropy(mode="vector", normalized=True, resolution=SIL.resolution, sample_range=SIL.sample_range)

In [ ]:
es_periodic = ES.fit_transform(DS.fit_transform([np.array(p) for p in pairs_periodic]))
es_regular = ES.fit_transform(DS.fit_transform([np.array(p) for p in pairs_regular]))

In [ ]:
plt.figure(figsize=(22,6))
plt.subplot(1,3,1)
plt.title("$H_0$")
plt.plot(x, es_periodic[0], label="periodic")
plt.plot(x, es_regular[0], label="regular")
plt.xlim(0, 0.12)
plt.legend(frameon=True)

plt.subplot(1,3,2)
plt.title("$H_1$")
plt.plot(x, es_periodic[1], label="periodic")
plt.plot(x, es_regular[1], label="regular")
plt.xlim(0, 0.18)

plt.subplot(1,3,3)
plt.title("$H_2$")
plt.plot(x, es_periodic[2], label="periodic")
plt.plot(x, es_regular[2], label="regular")
plt.xlim(0.05, 0.2)
plt.show()

per_halfmax = x[np.argmin(np.abs(es_periodic[0] - 0.5*es_periodic[0][0]))] / lbar
reg_halfmax = x[np.argmin(np.abs(es_regular[0] - 0.5*es_regular[0][0]))] / lbar
per_peak = x[np.argmax(es_periodic[1:], axis=1)] / lbar
reg_peak = x[np.argmax(es_regular[1:], axis=1)] / lbar
print(f"Periodic:  H_0 halfmax: {per_halfmax:.3f} lbar\t H_1 peak: {per_peak[0]:.3f} lbar\t H_2 peak: {per_peak[1]:.3f} lbar")
print(f"Regular:   H_0 halfmax: {reg_halfmax:.3f} lbar\t H_1 peak: {reg_peak[0]:.3f} lbar\t H_2 peak: {reg_peak[1]:.3f} lbar")

Look at full distribution of regular curves

In [ ]:
es_samples = [ES.fit_transform(DS.fit_transform(p)) for p in pairs_samples]
es_reg_mean = [np.mean(e, axis=0) for e in es_samples]
es_reg_std = [np.std(e, axis=0) for e in es_samples]

In [ ]:
plt.figure(figsize=(22,6))
plt.subplot(1,3,1)
plt.title("$H_0$")
plt.plot(x, es_periodic[0], label="periodic")
plt.plot(x, es_reg_mean[0], label="regular")
plt.fill_between(x, es_reg_mean[0]-es_reg_std[0], es_reg_mean[0]+es_reg_std[0], alpha=0.3, color='g')
plt.xlim(0, 0.12)
plt.legend(frameon=True)

plt.subplot(1,3,2)
plt.title("$H_1$")
plt.plot(x, es_periodic[1], label="periodic")
plt.plot(x, es_reg_mean[1], label="regular")
plt.fill_between(x, es_reg_mean[1]-es_reg_std[1], es_reg_mean[1]+es_reg_std[1], alpha=0.3, color='g')
plt.xlim(0, 0.18)

plt.subplot(1,3,3)
plt.title("$H_2$")
plt.plot(x, es_periodic[2], label="periodic")
plt.plot(x, es_reg_mean[2], label="regular")
plt.fill_between(x, es_reg_mean[2]-es_reg_std[2], es_reg_mean[2]+es_reg_std[2], alpha=0.3, color='g')
plt.xlim(0.05, 0.2)
plt.show()

per_halfmax = x[np.argmin(np.abs(es_periodic[0] - 0.5*es_periodic[0][0]))] / lbar
reg_halfmax = x[np.argmin(np.abs(es_reg_mean[0] - 0.5*es_reg_mean[0][0]))] / lbar
per_peak = x[np.argmax(es_periodic[1:], axis=1)] / lbar
reg_peak = x[np.argmax(es_reg_mean[1:], axis=1)] / lbar
print(f"Periodic:  H_0 halfmax: {per_halfmax:.3f} lbar\t H_1 peak: {per_peak[0]:.3f} lbar\t H_2 peak: {per_peak[1]:.3f} lbar")
print(f"Regular:   H_0 halfmax: {reg_halfmax:.3f} lbar\t H_1 peak: {reg_peak[0]:.3f} lbar\t H_2 peak: {reg_peak[1]:.3f} lbar")

### Error as function of $N$

As $N$ increases, boundary effects should get smaller, so we expect the non-periodic triangulation to get closer to the periodic one.

In [ ]:
N_vals = np.logspace(2.2, 4.6, 20, dtype=int)
lbar = boxsize / np.cbrt(N_vals)

points = [np.random.rand(N, 3) for N in N_vals]
pairs_periodic = [calc_persistence(data, boxsize=1) for data in points]
sts = [gd.AlphaComplex(points=data).create_simplex_tree() for data in points]
for sti in sts: sti.compute_persistence()
pairs_reg = [[np.sqrt(sti.persistence_intervals_in_dimension(d)) for d in range(3)] for sti in sts]

In [ ]:
es_reg = [ES.fit_transform(DS.fit_transform([p[i] for p in pairs_reg])) for i in range(3)]
es_per = [ES.fit_transform(DS.fit_transform([np.array(p[i]) for p in pairs_periodic])) for i in range(3)]

scale curves to be sef-similar and interpolate onto a common grid

In [ ]:
x_interp = np.linspace(0, 1.75, 500)
interp_reg = [[interpolate.interp1d(x/lbar[i], es_reg[d][i]/np.max(es_reg[d][i]), kind="slinear", bounds_error=False, fill_value=0)(x_interp) for i in range(len(N_vals))] for d in range(3)]
interp_per = [[interpolate.interp1d(x/lbar[i], es_per[d][i]/np.max(es_per[d][i]), kind="slinear", bounds_error=False, fill_value=0)(x_interp) for i in range(len(N_vals))] for d in range(3)]

In [ ]:
i = -1
for d in range(3):
    plt.plot(x_interp, interp_reg[d][i], 'b')
    plt.plot(x_interp, interp_per[d][i], 'g')
plt.show()

In [ ]:
l2_errors = [[np.linalg.norm(interp_reg[d][i] - interp_per[d][i]) for i in range(len(N_vals))] for d in range(3)]
l1_errors = [[np.linalg.norm(interp_reg[d][i] - interp_per[d][i], ord=1) for i in range(len(N_vals))] for d in range(3)]

In [ ]:
plt.figure(figsize=(14,6))
plt.subplot(1,2,1)
for d in range(3):
    plt.plot(N_vals, l2_errors[d], 'o', label=f"$H_{d}$")
plt.loglog()
plt.xlabel("$N$ points")
plt.ylabel("L2 norm (regular - periodic)")
plt.legend(frameon=True)

plt.subplot(1,2,2)
for d in range(3):
    plt.plot(N_vals, l1_errors[d], 'o', label=f"$H_{d}$")
plt.loglog()
plt.xlabel("$N$ points")
plt.ylabel("L1 norm (regular - periodic)")
plt.legend(frameon=True)
plt.show()

As $N$ increases, the non-periodic triangulation becomes more similar to the periodic one.